In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Construct list of election years
year=[]
for i in range(16):
    year.append(1960+i*4)
print(year)

#Initialise output list
state_results_list = []

#Iterate over years
for i in year:
    print(i)
    election_year = i
    
    #Import html for that year's article
    election_URL="https://en.wikipedia.org/wiki/"+str(election_year)+"_United_States_presidential_election"
    print(election_URL)
    election_response = requests.get(election_URL)
    election_html = election_response.content
    #Convert to string
    election_html_str = election_html.decode('utf-8')
    #Split to relevant html section
    result_html = election_html_str.split('<span class="mw-headline" id="Results_by_state">',1)
    result_html = result_html[1]
    #Format using BeautifulSoup
    election_soup = BeautifulSoup(result_html, "html.parser")
#    print(election_soup)
#    result_table = election_soup.find('table', attrs={'class':'wikitable sortable'})
    
    #Search for tables with sortable class
    result_table = election_soup.find('table', attrs={'class':['wikitable sortable','wikitable sortable sort-under']})
    #Isolate rows
    table_rows = result_table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            res.append(row)
    #Convert to DataFrame
    result_df = pd.DataFrame(res)
#    print(result_df)

    ##Identifies Positions of Republican and Democrat % columns

    table_headers_html = result_html.split("wikitable sortable",2)
#    print(result_html)
    headers_details_html = table_headers_html[1].split('colspan',10)
    iteration_number = 0
    dem = 0
    rep = 0
    #Initialise loop terminator
    matched=0
    for i in headers_details_html:
        if "Democrat" in i:
            dem=iteration_number
            matched+=1
        if "Republican" in i:
            rep=iteration_number
            matched+=1
        #Define loop termination condition (both Dem & Rep column found)
        if matched == 2:
            break
        iteration_number+=1
    print("dem",dem,"rep",rep)
    #Return stripped-down table body with State and Dem/Rep %s
    
    #Use structure for 2012 onwards
    if election_year>=2012:
        dem_col = dem*3-1
        rep_col = rep*3-1
    #Match 1992 structure
    elif election_year==1992:
        dem_col = (dem-5)*3
        rep_col = (rep-5)*3
    #Use structure for all else - 1960-1988, 1996-2008
    else:
        dem_col = dem*3-3
        rep_col = rep*3-3
    #Cut down irrelevant columns + reformat
    stripped_result_df = result_df[[0,dem_col,rep_col]]
    stripped_result_df = stripped_result_df.rename(columns={0:"State",dem_col:"Democrat",rep_col:"Republican"})
    #Append election_year to table
    stripped_result_df['Year']=election_year
    #Add iteration result to overall list
    state_results_list.append(stripped_result_df)

#Concatenate output list to a single DataFrame
state_all_results = pd.concat(state_results_list)

#Collate state values
state_remapping = state_all_results.copy()
state_mapping = {
   'Ala.':'Alabama',
    'Alab.':'Alabama',
    'Ariz.':'Arizona',
    'Arizona*':'Arizona',
    'Ark.':'Arkansas',
    'Calif.':'California',
    'Colo.':'Colorado',
    'Conn.':'Connecticut',
    'D. C.':'D.C.',
    'District of Columbia':'D.C.',
    'District of ColumbiaDistrict of Columbia':'D.C.',
    'Fla.':'Florida',
    'Ga.':'Georgia',
    'Ill.':'Illinois',
    'Ind.':'Indiana',
    'Kan.':'Kansas',
    'Ky.':'Kentucky',
    'La.':'Louisiana',
    'Maine â€ ':'Maine',
    'Maineâ€':'Maine',
    "Maine's 1st":'Maine-1',
    "Maine's 2nd":'Maine-2',
    'Mass.':'Massachusetts',
    'Md.':'Maryland',
    "ME-1Tooltip Maine's 1st congressional district":'Maine-1',
    "ME-2Tooltip Maine's 2nd congressional district":'Maine-2',
    'Mich.':'Michigan',
    'Minn.':'Minnesota',
    'Miss.':'Mississippi',
    'Mo.':'Montana',
    'Mont.':'Montana',
    'N.C.':'North Carolina',
    'N.D.':'North Dakota',
    'N.H.':'New Hampshire',
    'N.J.':'New Jersey',
    'N.J.[r]':'New Jersey',
    'N.M.':'New Mexico',
    'N.Y.':'New York',
    "NE-1Tooltip Nebraska's 1st congressional district":'Nebraska-1',
    "NE-2Tooltip Nebraska's 2nd congressional district":'Nebraska-2',
    "NE-3Tooltip Nebraska's 3rd congressional district":'Nebraska-3',
    'Neb. â€ ':'Nebraska',
    'Nebr. â€ ':'Nebraska',
    'Nebraska â€ ':'Nebraska',
    'Nebraskaâ€ ':'Nebraska',
    "Nebraska's 1st":'Nebraska-1',
    "Nebraska's 2nd":'Nebraska-2',
    "Nebraska's 3rd":'Nebraska-3',
    'Nev.':'Nevada',
    'Nev.[q]':'Nevada',
    'New Jersey[121]':'New Jersey',
    'New York[122]':'New York',
    'Ohio[123]':'Ohio',
    'Okla.':'Oklahoma',
    'Ore.':'Oregon',
    'Pa.':'Pennsylvania',
    'R.I.':'Rhode Island',
    'S.C.':'South Carolina',
    'S.D.':'South Dakota',
    'Tenn.':'Tennessee',
    'Texas[s]':'Texas',
    'Va.':'Virginia',
    'Vt.':'Vermont',
    'W.Va.':'West Virginia',
    'Wash.':'Washington',
    'Wis.':'Wisconsin',
    'Wisconsin[124]':'Wisconsin',
    'Wyo.':'Wyoming'
}
#Apply State mapping
state_remapping['State_Mapped'] = state_remapping['State'].map(state_mapping)
#Merge State_Mapped and State, prioritising Mapped version.
state_remapping['State'] = state_remapping['State_Mapped'].fillna(state_remapping['State'])
state_remapping=state_remapping.drop(columns=['State_Mapped'])

#Extract values from Dem/Rep columns
parties = ['Democrat','Republican']
for i in parties:
    state_remapping[i] = state_remapping[i].str.extract(r'([\d\.]+)')
state_remapping

#Filter out incorrectly formatted "US" rows
extra_edits = state_remapping.copy()
extra_edits = extra_edits[extra_edits['State']!='US']

#Import yearly percentages
aggregate_results_df = pd.read_csv('1976-2020-president.csv')

#Get % of yearly vote by party
#Aggregate by year & party
votes_by_yearparty = aggregate_results_df.groupby(['year','party_simplified'])['candidatevotes'].sum()
#Aggregate by year
votes_by_year = aggregate_results_df.groupby('year')['candidatevotes'].sum()
percentage_votes_by_year_party = (votes_by_yearparty / votes_by_year).reset_index()

#Crosstab results to one row per year
pivoted_results = percentage_votes_by_year_party.pivot_table(index='year', columns='party_simplified', values='candidatevotes', aggfunc='sum').reset_index()

#Rename columns
pivoted_results.columns.name = None #Remove axis
pivoted_results.columns = ['Year', 'Democrat', 'Libertarian', 'Other', 'Republican']

#Match state-level format
pivoted_results['Democrat'] *= 100
pivoted_results['Republican'] *= 100
pivoted_results['State'] = "General Election"
pivoted_results = pivoted_results[['State', 'Democrat', 'Republican', 'Year']]

#Add General Election results to State-level
combined_df = pd.concat([extra_edits, pivoted_results], ignore_index=True)

#Write to csv
combined_df.to_csv('GeneralElectionResults.csv',index=True)

[1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
1960
https://en.wikipedia.org/wiki/1960_United_States_presidential_election
num 3 dem 2 rep 3
1964
https://en.wikipedia.org/wiki/1964_United_States_presidential_election
num 3 dem 2 rep 3
1968
https://en.wikipedia.org/wiki/1968_United_States_presidential_election
num 3 dem 3 rep 2
1972
https://en.wikipedia.org/wiki/1972_United_States_presidential_election
num 3 dem 3 rep 2
1976
https://en.wikipedia.org/wiki/1976_United_States_presidential_election
num 3 dem 2 rep 3
1980
https://en.wikipedia.org/wiki/1980_United_States_presidential_election
num 3 dem 3 rep 2
1984
https://en.wikipedia.org/wiki/1984_United_States_presidential_election
num 3 dem 3 rep 2
1988
https://en.wikipedia.org/wiki/1988_United_States_presidential_election
num 3 dem 3 rep 2
1992
https://en.wikipedia.org/wiki/1992_United_States_presidential_election
num 7 dem 6 rep 7
1996
https://en.wikipedia.org/wiki/1996_United_States_pre